In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
# just in case reload_ext does not work
# Remove the modules if they exist in sys.modules
for mod in ["typst_importer.typst_to_svg", "typst_importer.curve_utils", "typst_importer.svg_preprocessing"]:
    if mod in sys.modules:
        print(f"Removing {mod} from sys.modules")
        del sys.modules[mod]

# Now Python 'forgets' those imports. Reimport them:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

In [642]:
import bpy
import sys
import tempfile
import typst
from pathlib import Path

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))

from typst_importer.notebook_utils import display_svg

#uv pip install lxml
#uv pip install numpy==1.26.4
# so that code highlighting works

temp_dir = Path(tempfile.gettempdir())
#temp_dir = Path.cwd()
svg_file = temp_dir / "step1.svg"

# Select Typst File

In [ ]:
file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

$ a= b/c $  
"""
typst_file = temp_dir / "step1.typ"
typst_file.write_text(file_content)

In [644]:
typst_path = Path.home() / "projects/blender_typst_importer/docs/"
typst_file = typst_path / "matrix.txt"
typst_file = typst_path / "code.txt"
typst_file = typst_path / "color_eq.txt"

# Manual SVG conversion + add to scene

In [ ]:
from typst_importer.svg_preprocessing import preprocess_svg, stroke_to_filled_path

typst.compile(typst_file, format="svg", output=str(svg_file))

step1_content = svg_file.read_text()
step2_content = preprocess_svg(step1_content)
step3_content = stroke_to_filled_path(step2_content)


display_svg(step1_content , width='500px')
display_svg(step3_content , width='500px')

# SVG Blender Operations

In [646]:
output_file = temp_dir / "step3.svg"
output_file.write_text(step3_content)

bpy.ops.import_curve.svg(filepath=str(output_file))
col = bpy.context.scene.collection.children['step3.svg']
col.name = "Formula"

for obj in col.objects:
    obj.scale = (100, 100, 100) # this is the old method, #better 
    # obj.data.transform(Matrix.Scale(scale_factor, 4))

In [647]:
# Loop through each object in the collection and set its origin
bpy.ops.object.select_all(action='DESELECT')
if col.objects:
    # Set the first object as active
    bpy.context.view_layer.objects.active = col.objects[0]
    # Now we can safely set the mode to OBJECT
    bpy.ops.object.mode_set(mode='OBJECT') 
    for obj in col.objects:
        bpy.context.view_layer.objects.active = obj
        obj.select_set(True)
        bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='MEDIAN')
        obj.select_set(False)

# Half automated



In [648]:
import tempfile
from pathlib import Path
from typst_importer.typst_to_svg import typst_to_blender_curves
typst_file =  Path(tempfile.gettempdir()) / "hello.typ"

header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ a= b/c $"
typst_file.write_text(header+body)
collection = typst_to_blender_curves(typst_file)

# Full automated

In [ ]:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

c = typst_express("$ a = b/d $", scale_factor=100, origin_to_char=False)

[I 2025-02-07 11:50:49.339 ServerApp] 302 GET / (@127.0.0.1) 0.42ms
[I 2025-02-07 11:50:49.344 LabApp] 302 GET /lab? (@127.0.0.1) 2.59ms
[I 2025-02-07 11:50:49.406 ServerApp] Connecting to kernel 5e4cc335-0fd3-4d22-b99a-fc5bc33a00cb.
[I 2025-02-07 11:50:49.408 ServerApp] Discarding 139 buffered messages for 5e4cc335-0fd3-4d22-b99a-fc5bc33a00cb:814013a2-a002-4321-9b93-31f97d0afaf9
